# **ACCENTing Transcribe’s Accuracy**
*by 
Ethan Feldman, Elayna Goodman, Alvin Huezo, Adrian Lee, Bo Lee, Anna Ringwood*



In [ ]:
%%html
<marquee style='width: 70%; color: hotpink;'><b>Team Rustica</b></marquee

<img src=https://bilingua.io/wp-content/uploads/2017/05/hello-in-100-languages.jpg, width = "500"> 

###**Overview of Amazon Transcribe**

Amazon Transcribe is a machine learning service available on Amazon Web Services, or AWS. This service has many features, the most prominent of which is, as its name suggests, to create a text transcription from an audio file. In addition to creating transcriptions from uploaded audio files, AWS Transcribe can:

*   provide real-time transcriptions
*   generate timestamps for each word
*   account for domain-specific words and technical terminology to increase accuracy
*   recognize multiple speakers
*   transcribe and identify multiple audio channels
*   detect the language being spoken in the audio
*   automatically redact personally identifiable information (PII) from the transcripts generated <br>

Click [here](https://aws.amazon.com/transcribe/?nc=sn&loc=0) for more information on Transcribe and its uses and clients. There is also a [guide](http://example-350.s3-website-us-east-1.amazonaws.com) created by Team Rustica that gives a walkthrough on using this service to transcribe a video. <br>
![AWS Transcribe](https://miro.medium.com/max/400/0*BNicZbcGBnG5t2qX.png) <br>
Interestingly, Transcribe also has a version that is aimed towards serving the medical field, Transcribe Medical, which is equipped to accurately transcribe medicine names, diseases, and procedures for a wide variety of uses (doctor-patient phone calls, documentation, efficiently submitting prescriptions, and similar services). <br>

As can be seen, there are many uses for Amazon Transcribe that can be applied to a myriad of settings, proving its value and functionality. 

###**Putting Amazon Transcribe to the Test**

Given what we know about AWS Transcribe, we wanted to measure how accurate the service is when converting speech-to-text in different accents. *We hypothesized that countries with a greater percentage of English speakers would have greater rates of accuracy translation in Transcribe.* Our reasoning behind this was that as Amazon continues to expand and connect with more countries that speak English, the company must adapt its service to the people using it. <br>

<img src=https://youmatter.world/app/uploads/sites/2/2019/01/globalization-definition-benefits-effects-examples.jpg, width = "500"> <br>

We believe this topic is important for two major reasons. One, as the world globalizes, Transcribe will have to improve its handling of speakers with different accents. Having an accurate service will allow for clear and efficient communication when discussing ideas and having general conversations. Furthermore, closed captioning is an important service for many, especially for those who have hearing impairments. The deaf community, in fact, recently had a movement calling for [#NoMoreCraptions](https://www.theatlantic.com/health/archive/2019/08/youtube-captions/595831/) because of the sub-par quality of closed captioning on the popular streaming platform YouTube. 

###**Formulating Our Test**

In order to better understand when Transcribe may fail, we aimed to find five varying speakers/accents to test the accuracy of the service. To add progressive difficulty for Transcribe, we also found three scripts with words of varying difficulty that could incrementally challenge the machine learning model. <br>

One of the greatest difficulties of this study was to accurately capture different types of accents, especially since all Team Rustica members have a more “conventional” American accent. In order to get a consistent reading of widely different accents, we used [ttsreader.com](https://ttsreader.com/), a text-to-speech website that reads transcripts with different accents and speakers. This allowed us to base accents around certain geographic regions. We chose countries by finding out [how much of the native population actually speaks English](https://en.wikipedia.org/wiki/List_of_countries_by_English-speaking_population). In this particular experiment we used accents from the following areas: 

*   United States
*   United Kingdom
*   China
*   Spain
*   India

It is important to note that we used only female voices to recite these scripts as a control.

There are major variations in the percentage of people who speak English in these selected countries. The population statistics found are reported below:

![Pic](https://drive.google.com/uc?id=1QKTw4ovdlLFdYFhNTyrnaAzajidXewXq) <br>

---
The scripts that we used were excerpts from the following:


*   "Easy" Difficulty - [A Pep Talk from Kid President](http://complicatedmelody.com/content/transcript-pep-talk-kid-president) by Kid President (SoulPancake)
*   "Medium" Difficulty - [Inside the Mind of a Master Procrastinator](https://www.ted.com/talks/tim_urban_inside_the_mind_of_a_master_procrastinator/transcript#t-25363) by Tim Urban (TedTalk)
*   "Hard" Difficulty - [The ability to estimate knowledge and performance in college: A metacognitive analysis](https://doi.org/10.1023/A:1003040130125) by Howard T. Everson & Sigmund Tobias <br>

The Soulpancake video contains language that should be easily interpretable by someone with at least an elementary school education, and the TED Talk should easily reach middle school and high school students. The last passage should be the most difficult to interpret as the college-level vocabulary may be used less frequently than the language in the two other scripts. Rather than using the whole video/article, we shortened the excerpts to approximately 1.5-2 minutes long. That way, the transcription on the part of Transcribe would not be too lengthy, but we would also still have a sufficient amount of words to assess accuracy upon.

###**Project Architecture**

To understand our data collection process, we will consider the following architecture diagram:

![architecture](https://drive.google.com/uc?id=1w-rVGM3eNTq9KFI8EyAQgluPsLvZlNmC)

The original transcripts that we used are accessible in our first S3 bucket. These files would be available to any user who might want more information about the data that we used.

The transcripts from the “Actual-Transcripts” bucket are run through [ttsreader.com](https://ttsreader.com), and the audio files generated from this service are available in the second bucket.

We then ran the sound files through Amazon Transcribe, and the newly automated captions were made available in the third bucket. These files were used for data analysis in Amazon Sagemaker.

## **Implementing Transcribe**

In order to use Transcribe within Sagemaker, we followed the process [here](http://audio-files-to-be-transcribed.s3-website-us-east-1.amazonaws.com/). Below is a snippet of the code necessary to convert audio files into transcripts.

In [ ]:
import boto3
transcribe = boto3.client('transcribe')

job_name = "American_Medium_Transcript"
job_uri = "https://audio-files-to-be-transcribed.s3.amazonaws.com/American+Medium+Audio+Extracted+2.wav"

In this example, we instructed Amazon Sagemaker to look for a file in the "Audio" S3 Bucket. In this bucket, we acquired the file that contains the sound bite of the American-accented female voice reading back the TED Talk transcript.

In [ ]:
transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
    MediaFormat='wav',
    LanguageCode='en-US',
    OutputBucketName='aws-generated-transcripts'
)

Then, we begin the transcription job. 

`TranscriptionJobName`: the name of this particular transcription job, which is the TED Talk in the American accent in this case. 

`Media`: the media file to be transcribed, which is the actual audio file containing the speaker's rendition of the passage.

`MediaFormat`: the format of the media file which is in .wav.

`LanguageCode`: the language of the audio file. We want English in this analysis.

`OutputBucketName`: the S3 bucket in which we want the transcript to be stored, in this case, `aws-generated-transcripts`. <br>

The remaining audio files were generated into transcripts in this manner.

###**Process of Data Analysis**

To analyze the data that we compiled, we used the Levenshtein distance as well as a review of Transcribe's confidence on its transcription of the audio based on the length of the word.


---


**Levenshtein Distance Observations**

To analyze our data, we found the [Levenshtein distance](https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a), which measures the difference between two sequences. Specifically, it quantifies the minimum number of single-character edits needed to change one sequence into another. A higher Levenshtein distance, therefore, would indicate more edits needed and lower accuracy. After determining the Levenshtein distance for each generated transcripted to its respective original transcript, a visual was made for the data.

![Levenshtein](https://drive.google.com/uc?id=19jpOaiao51xxajyPYUTQHJh23iHSpm8f)

As can be seen from the graph, the easy difficulty transcripts had much lower Levenshtein distances than that of the medium and hard transcripts across all five accents. Additionally, the medium difficulty transcripts had less error than the hard difficulty transcripts, though this difference is not large. As far as general trends go, the figure indicates that AWS Transcribe made more mistakes with more difficult transcripts.


Looking within the easy difficulty transcripts, the Spanish accent had the highest Levenshtein distance, followed by the Chinese accent. The other accents, American, British, and Hindi, had similar distances. Therefore, AWS Transcribe made more mistakes with the Spanish and Chinese accents. Using specific Levenshtein distances, considering just the easy transcripts, the accents ranked from most to least accurate were: American (90), British (98), Hindi (114), Chinese (221), Spanish (308). <br>


---


**Assessing Transcribe's Confidence Based on Word Length**

Amazon Transcribe provides information about how confident the program feels about a word the accuracy of a word that it attempts to transcribe. A higher confidence score means that the service is more certain about what it just "heard." After determining the confidence for each generated transcripted and finding the mean and standard deviation at each word length, a visual was made for the data.

<img src="https://drive.google.com/uc?id=1xI0E-lAIIGq_ARevecEYVFzvYRFibaFG" alt="drawing" width="3000"/>

Across all five accents, there tends to be a decrease in confidence as the word length increases, indicated by the negative sloping regressions. This negative relationship is stronger in the American accent and the Spanish accent; additionally, the Spanish accent begins and ends with lower confidence than that of the other four accents. This indicates that AWS Transcribe is less confident that the service is accurate in its transcription as the length of the word gets larger.

Also, the standard deviation decreases until about word length 4 to 5, then increases again beyond that for the five accents. Short words are therefore transcribed with less confidence variation, but words that are too short (less than 4 letters) and words that are long (more than 5 letters) have greater variation in confidence. 


It is important to note that the y-axis begins at 0.75 confidence instead of the conventional 0 in order to better visualize the data. This also serves to let us know that the service is always at least 70% certain that it has chosen the right word. Additionally, the x-axis does not have any entries at "0" because there are no words with a "zero-length."


---

To access the code that contains the process used to compute the Levenshtein distance and the word length confidence assessment, please click [here](http://aws-generated-transcripts.s3-website-us-east-1.amazonaws.com/). This link also contains the data analysis for the Levenshtein distance and word confidence as well as the corresponding visualizations. 

###**Conclusion**

We were able to successfully make a transcript out of the audio files using AWS Transcribe, and analyze their accuracy using the Levenshtein distance. We also examined how confident the machine learning service was on the transcribed words based on word length. The data obtained did not support the hypothesis, however, as a greater percentage of English speakers in a country did not lead to having greater rates of accuracy translation in Transcribe. Mainly, Transcribe had a relatively harder time transcribing the Spanish and Chinese accents in the easy script, and had difficulty transcribing in general in the medium and hard scripts. There was also less confidence in the transcriptions for longer words.

There were, of course, limitations to this process. Specifically, the text-to-speech reader had different levels of accent thickness. For example, the Spanish accent was much thicker than the Hindi accent, which may have been why the Spanish accent had the least accuracy and least confidence. In addition, the amount of punctuations in the scripts, such as commas and question marks, may have influenced the tone and cadence at which the scripts were read. 

Although the service is not perfect, advancements are continuously being made. It would be productive, however, to continue finding flaws in Transcribe in order to improve it. More accents can be used, for instance, to determine where Transcribe falls flat in transcribing speech. In the end, there are many advantages for AWS Transcribe, and it is a very useful tool with important implications.

Feel free to visit our [Github Repo](https://github.com/QTM350/ACCENTing-AWS_Transcribe-Accuracy-Code) for more details about this project!


###**Sources**

Websites:
*   AWS. "Amazon Transcribe." https://aws.amazon.com/transcribe/?nc=sn&loc=0
*   Besner, Linda. "When Is a Caption Close Enough?" https://www.theatlantic.com/health/archive/2019/08/youtube-captions/595831/
*   Diana. "Transcript: A Pep Talk From Kid President." http://complicatedmelody.com/content/transcript-pep-talk-kid-president
*   Everson, Howard T., Tobias, Sigmund. "The ability to estimate knowledge and performance in college: A megacognitive analysis." https://link.springer.com/article/10.1023/A:1003040130125
*   List from Wikipedia. "List of countries by English-speaking population." https://en.wikipedia.org/wiki/List_of_countries_by_English-speaking_population
*   Radecic, Dario. "Calculating String Similarity in Python." https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a
*   Speech-to-text. https://ttsreader.com/
*   Urban, Tim. "Inside the mind of a master procrastinator." https://www.ted.com/talks/tim_urban_inside_the_mind_of_a_master_procrastinator/transcript#t-25363 <br>

Images:
*   Amazon Transcribe. https://miro.medium.com/max/400/0*BNicZbcGBnG5t2qX.png
*   Globalization. https://youmatter.world/app/uploads/sites/2/2019/01/globalization-definition-benefits-effects-examples.jpg
*   Hello in Languages. https://bilingua.io/wp-content/uploads/2017/05/hello-in-100-languages.jpg





